In [1]:
pip install topic-wizard

  Using cached topic_wizard-0.5.0-py3-none-any.whl (108 kB)
  Using cached dash_extensions-1.0.12-py3-none-any.whl (1.4 MB)
  Using cached dash_iconify-0.1.2-py3-none-any.whl (18 kB)
  Using cached dash_mantine_components-0.12.1-py3-none-any.whl (500 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached pandas-1.5.3-cp311-cp311-macosx_10_9_x86_64.whl (11.9 MB)
  Using cached umap-learn-0.5.5.tar.gz (90 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached wordcloud-1.9.3-cp311-cp311-macosx_10_9_x86_64.whl (172 kB)
  Using cached Flask_Caching-2.1.0-py3-none-any.whl (28 kB)
  Using cached dash-2.15.0-py3-none-any.whl (10.2 MB)
  Using cached dataclass_wizard-0.22.3-py2.py3-none-any.whl (90 kB)
  Using cached jsbeautifier-1.14.11.tar.gz (75 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ..

In [2]:
from sklearn.datasets import fetch_20newsgroups

corpus = fetch_20newsgroups(subset="all").data

In [3]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# Creating a bag-of-words vectorizer
# We cut low and high frequency words and filter out English stopwords
vectorizer = CountVectorizer(min_df=10, max_df=0.5, stop_words="english")
# We create a topic model with ten topics
topic_model = NMF(n_components=10)
# Then we set up a pipeline
topic_pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("topic_model", topic_model),
    ]
)

In [4]:
topic_pipeline.fit(corpus)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.5, min_df=10, stop_words='english')),
                ('topic_model', NMF(n_components=10))])

In [5]:
import topicwizard
topicwizard.visualize(pipeline=topic_pipeline, corpus=corpus)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preprocessing


In [6]:
topicwizard.load(filename="topic_data.joblib")

Preparing data


FileNotFoundError: [Errno 2] No such file or directory: 'topic_data.joblib'

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:92: RuntimeWarning:

divide by zero encountered in log

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:94: RuntimeWarning:

divide by zero encountered in log

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:92: RuntimeWarning:

divide by zero encountered in log

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:94: RuntimeWarning:

divide by zero encountered in log

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:92: RuntimeWarning:

divide by zero encountered in log

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/topicwizard/prepare/topics.py:94: RuntimeWarning:

divide by zero encountered in log

/Lib